In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.legacy import Adam as LegacyAdam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import accuracy_score

# Load CSV file with image_id and Male columns
celeba_data = pd.read_csv('clean_attribute_lila2.csv')
celeba_data['male'] = celeba_data['male'].astype(str)

# Split the data into training and validation sets
train_data, valid_data = train_test_split(celeba_data, test_size=0.1, random_state=42)

# Set up data generators
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_dataframe(
    dataframe=train_data,
    directory='Images',
    x_col='file_name',
    y_col='male',
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode='binary',
    target_size=(224, 224)
)

valid_generator = datagen.flow_from_dataframe(
    dataframe=valid_data,
    directory='Images',
    x_col='file_name',
    y_col='male',
    batch_size=32,
    seed=42,
    shuffle=False,
    class_mode='binary',
    target_size=(224, 224)
)

# Define hyperparameter values to be tuned
depth_values = [18, 34, 50]
kernel_size_values = [(3, 3), (5, 5)]
dropout_rate_values = [0.2, 0.4, 0.6]
l2_reg_values = [1e-4, 1e-3, 1e-2]
learning_rate_values = [0.001, 0.01, 0.1]
batch_size_values = [32, 64, 128]
optimizer_values = ['adam', 'sgd', 'rmsprop']

# Loop for hyperparameter tuning
for depth in depth_values:
    for kernel_size in kernel_size_values:
        for dropout_rate in dropout_rate_values:
            for l2_reg in l2_reg_values:
                for learning_rate in learning_rate_values:
                    for batch_size in batch_size_values:
                        for optimizer in optimizer_values:
                            # Load ResNet101 model without the top layer
                            base_model = ResNet101(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

                            # Freeze the layers of the pre-trained model
                            for layer in base_model.layers:
                                layer.trainable = False

                            # Create a new model on top of ResNet101
                            model = Sequential()
                            model.add(base_model)
                            model.add(GlobalAveragePooling2D())
                            model.add(Dense(128, activation='relu'))
                            model.add(Dense(1, activation='sigmoid'))

                            # Set up optimizer
                            if optimizer == 'adam':
                                optimizer = Adam(learning_rate=learning_rate)
                            elif optimizer == 'rmsprop':
                                optimizer = RMSprop(learning_rate=learning_rate)
                            elif optimizer == 'sgd':
                                optimizer = SGD(learning_rate=learning_rate)

                            # Compile the model
                            model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

                            # Implement ReduceLROnPlateau and EarlyStopping callbacks
                            reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-6, verbose=1)
                            early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

                            # Train the model
                            history = model.fit(
                                train_generator,
                                epochs=10,
                                validation_data=valid_generator,
                                callbacks=[reduce_lr, early_stop]
                            )

                            # Save the trained model to an h5 file
                            model.save(f"ResNet101_Tuned_Depth{depth}_Kernel{kernel_size}_Dropout{dropout_rate}_L2{str(l2_reg)}_LR{str(learning_rate)}_Batch{batch_size}_Opt{optimizer}.h5")

Found 4500 validated image filenames belonging to 2 classes.
Found 500 validated image filenames belonging to 2 classes.


Epoch 1/10
141/141 [==============================] - 186s 1s/step - loss: 0.6900 - accuracy: 0.5733 - val_loss: 0.6696 - val_accuracy: 0.5940 - lr: 0.0010
Epoch 2/10
141/141 [==============================] - 184s 1s/step - loss: 0.6493 - accuracy: 0.6196 - val_loss: 0.6316 - val_accuracy: 0.6420 - lr: 0.0010
Epoch 3/10
141/141 [==============================] - 185s 1s/step - loss: 0.6377 - accuracy: 0.6458 - val_loss: 0.6220 - val_accuracy: 0.6200 - lr: 0.0010
Epoch 4/10
141/141 [==============================] - 183s 1s/step - loss: 0.6099 - accuracy: 0.6702 - val_loss: 0.6041 - val_accuracy: 0.7460 - lr: 0.0010
Epoch 5/10
141/141 [==============================] - 183s 1s/step - loss: 0.5983 - accuracy: 0.6813 - val_loss: 0.5680 - val_accuracy: 0.6880 - lr: 0.0010
Epoch 6/10
141/141 [==============================] - 184s 1s/step - loss: 0.5756 - accuracy: 0.7064 - val_loss: 0.6041 - val_accuracy: 0.6520 - lr: 0.0010
Epoch 7/10
141/141 [==============================] - 184s 1s/st

/Users/lilakastara/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


NameError: name 'SGD' is not defined